In [1]:
import numpy as np
import re
import networkx as nx
import random
import matplotlib.pyplot as plt
import os
import glob
from ALB_instance_tools import *
from scenario_trees import *
import copy
from MALBP_instance_generation import *

## Making production sequences


In [2]:
MODEL_MIXTURES = {'A':0.34, 'B':0.33, 'C':0.33}
#mc_seq = monte_carlo_tree(5, MODEL_MIXTURES, enum_depth=0 , n_samples=5)
generate_tree_csv(monte_carlo_tree_limit, 5, MODEL_MIXTURES, enum_depth=0 , n_samples=5, filename='SALBP_benchmark/MM_instances/scenario_trees/5_takts_5_samples_3_models.csv')

Sampling  5 with seed  None
final sequences {0: {'sequence': ['B', 'B', 'A', 'B', 'B'], 'probability': 0.2}, 1: {'sequence': ['A', 'A', 'A', 'C', 'C'], 'probability': 0.2}, 2: {'sequence': ['B', 'C', 'C', 'C', 'C'], 'probability': 0.2}, 3: {'sequence': ['A', 'A', 'C', 'C', 'C'], 'probability': 0.2}, 4: {'sequence': ['C', 'C', 'B', 'A', 'A'], 'probability': 0.2}}
          sequence probability
0  [B, B, A, B, B]         0.2
          sequence probability
0  [A, A, A, C, C]         0.2
          sequence probability
0  [B, C, C, C, C]         0.2
          sequence probability
0  [A, A, C, C, C]         0.2
          sequence probability
0  [C, C, B, A, A]         0.2
writing to file  SALBP_benchmark/MM_instances/scenario_trees/5_takts_5_samples_3_models.csv
          sequence probability
0  [B, B, A, B, B]         0.2
1  [A, A, A, C, C]         0.2
2  [B, C, C, C, C]         0.2
3  [A, A, C, C, C]         0.2
4  [C, C, B, A, A]         0.2


## Making model instances

In [2]:
import yaml
NO_EQUIPMENT = 4
seed = 1
NO_WORKERS =4
NO_STATIONS = 2
WORKER_COST = 500
RECOURSE_COST = WORKER_COST * 2
TAKT_TIME = 500
SEQUENCE_LENGTH = 3
#MODEL_MIXTURES = {'A':0.34, 'B':0.33, 'C':0.33}
# MODEL_MIXTURES = {'A':0.60, 'B':0.40}

instances = [
    {'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_200.alb', 'name': 'A', 'probability': 0.6},
     #{'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_17.alb', 'name': 'C', 'probability': 0.4},
     {'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_16.alb', 'name': 'B', 'probability': 0.4}]
#mm_instance = MultiModelInstance( init_type='from_yaml',mm_yaml_file='/Users/letshopethisworks2/Documents/Phd Paper material/MMABPWW/SALBP_benchmark/MM_instances/n=20_1_n=20_2_n=20_3_.yaml', sequence_length=SEQUENCE_LENGTH, max_workers=NO_WORKERS, no_stations=NO_STATIONS, worker_cost=WORKER_COST, recourse_cost=RECOURSE_COST)


small_instances = get_instance_list('SALBP_benchmark/MM_instances/model_data', extension='.yaml')
instance_groups = pair_instances(instances, MODEL_MIXTURES)
mm_instance = MultiModelTaskTimesInstance(model_dicts=instances, takt_time=TAKT_TIME, sequence_length=SEQUENCE_LENGTH, max_workers=NO_WORKERS, no_stations=NO_STATIONS, worker_cost=WORKER_COST, recourse_cost=RECOURSE_COST)

print(mm_instance.name)
print(mm_instance.data)
print(mm_instance.takt_time)


mm_instance.genererate_task_times(change_func=change_task_times_linear)

mm_instance.model_data_to_yaml(f'SALBP_benchmark/MM_instances/model_data/')


save_MALBPW_config('small_instance', NO_WORKERS, NO_STATIONS, 
                   WORKER_COST, RECOURSE_COST, SEQUENCE_LENGTH, 
                   'SALBP_benchmark/MM_instances/small_instance_config.yaml', 
                   equipment_files =[ 'SALBP_benchmark/MM_instances/equipment_data/random_O20_E4_S4_seed42.yaml'], instance_files=small_instances )


In [3]:
#instance_groups = pair_instances(instances, MODEL_MIXTURES)

instance_list = read_instance_folder("SALBP_benchmark/small data set_n=20/")[3:6]
print(instance_list)
model_names = ['A', 'B',]
interval = (0.8, 0.8)


fp = 'SALBP_benchmark/debugging_ds/'
#if filepath does not exist, create it
if not os.path.exists(fp):
    os.makedirs(fp)
make_reduced_instances(fp,instance_list, model_names, takt_time=TAKT_TIME, sequence_length=SEQUENCE_LENGTH, max_workers=NO_WORKERS, no_stations=NO_STATIONS, worker_cost=WORKER_COST, recourse_cost=RECOURSE_COST, interval_choice = interval)

['SALBP_benchmark/small data set_n=20/instance_n=20_4.alb', 'SALBP_benchmark/small data set_n=20/instance_n=20_5.alb', 'SALBP_benchmark/small data set_n=20/instance_n=20_6.alb']
These are the model_dicts
[{'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_4.alb', 'name': 'A', 'probability': 0.4625029902264759}, {'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_5.alb', 'name': 'B', 'probability': 0.5374970097735241}]
This is the data
These are the model_dicts
[{'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_5.alb', 'name': 'A', 'probability': 0.5576601948202707}, {'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_6.alb', 'name': 'B', 'probability': 0.44233980517972926}]
This is the data


## Making equipment

In [5]:
equip = Equipment(20,8, 4,  seed=42, mean = 20, variance = 7)
print("number of stations", equip.no_stations)
print("number of tasks", equip.no_tasks)
equip.c_se
equip.to_yaml(f'SALBP_benchmark/MM_instances/equipment_data/')

doing new method
number of stations 4
number of tasks 20
